In [18]:
import mne
import numpy as np

import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression

# from mne.datasets import sample
from mne.decoding import LinearModel, GeneralizingEstimator
# from mne.decoding import cross_val_multiscore, LinearModel, GeneralizingEstimator, Scaler, \
#                          Vectorizer
# from sklearn.model_selection import StratifiedKFold, cross_val_score, StratifiedShuffleSplit, \
#                                     RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder ,StandardScaler
# from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

In [6]:
SAVE_EPOCH_ROOT = '../../data/version5.2/preprocessed/epochs/aft_ICA_rej/' #my laptop
filename_epoch = SAVE_EPOCH_ROOT +  'epochs_sec_applyBaseline_subj1-afterRejICA-epo.fif'

#Read Epochs
epochs_orig = mne.read_epochs(filename_epoch, proj=True, preload=True, verbose=None)
epochs = epochs_orig.copy()
subset = epochs['pred']['non'].copy()
subset = subset.pick_types(eeg=True)
subset.crop(tmin=-0.4,tmax=0.5)

Reading ../../data/version5.2/preprocessed/epochs/aft_ICA_rej/epochs_sec_applyBaseline_subj1-afterRejICA-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 129) active
    Found the data of interest:
        t =    -400.00 ...    5000.00 ms
        0 CTF compensation matrices available
1197 matching events found
Applying baseline correction (mode: mean)
Adding metadata with 16 columns
Created an SSP operator (subspace dimension = 1)
1 projection items activated


<EpochsFIF  |   876 events (all good), -0.4 - 0.5 sec, baseline [-0.4, 0], ~195.1 MB, data loaded, with metadata,
 'pred/down/non': 216
 'pred/left/non': 224
 'pred/right/non': 219
 'pred/up/non': 217>

## Some preprocessing

In [7]:
if subset['Block==6'].metadata.Ptrn_Type.values.shape[0]>0:
       main_ptrn = subset['Block==6'].metadata.Ptrn_Type.values[0]
else:
       main_ptrn = subset['Block==8'].metadata.Ptrn_Type.values[0]

# only early blocks
subset = subset['Block<7'].copy()
subset = subset['Block>2'].copy()

# Group data based on the previous trial
Grp1 = subset['Trgt_Loc_prev==1'].copy()
Grp2 = subset['Trgt_Loc_prev==2'].copy()
Grp3 = subset['Trgt_Loc_prev==3'].copy()
Grp4 = subset['Trgt_Loc_prev==4'].copy()

Grp1._data = (Grp1._data - np.mean(Grp1._data)) / np.std(Grp1._data)
Grp2._data = (Grp2._data - np.mean(Grp2._data)) / np.std(Grp2._data)
Grp3._data = (Grp3._data - np.mean(Grp3._data)) / np.std(Grp3._data)
Grp4._data = (Grp4._data - np.mean(Grp4._data)) / np.std(Grp4._data)

# apply decoder

In [19]:
le = LabelEncoder()
clf_SVC  = make_pipeline(
                        StandardScaler(),
                        LinearModel(LinearSVC(random_state=0, max_iter=10000)))
time_gen = GeneralizingEstimator(clf_SVC, scoring='roc_auc', n_jobs=1,verbose=True)

In [20]:
Grp_data = Grp1
X=Grp_data.copy()._data
y=le.fit_transform(Grp_data.copy().metadata.Trgt_Loc_main)

time_gen.fit(X=X ,y=y)
score_wocv = time_gen.score(X=X, y=y) #scores without cv
score_diag_wocv = np.diag(score_wocv)
scores_wocv = (score_wocv.copy(), score_diag_wocv.copy())

[...............] 100.00% Fitting GeneralizingEstimato
[........       ]  59.56% Scoring GeneralizingEstimato


KeyboardInterrupt: 